# Imports

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from __future__ import print_function

import glob
import math
import os
import copy

import numpy as np
import pandas as pd


from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Classes

In [ ]:
class NN:
    def __init__(self, input_shape, units, drop_prob=0):
        """
        units: (10, 11, 12)
            => 3 couches avec 10 neurones pour la premiere
        """
        nb_output = input_shape
        self.weights = []
        for nb_neurones in units:
            self.weights.append(np.random.rand(nb_output, nb_neurones) * 0.1)
            self.weights.append(np.zeros(nb_neurones))
            nb_output = nb_neurones
        
        # TODO: a finir
        self.drop_prob = drop_prob
        self.drop_out_layers = []

#==========================================================================================================
    
    def print_weights(self):
        for l in self.weights:
            print(l)

#==========================================================================================================

    def preactiv(self, x):
        res = x
        for i in range(0, len(self.weights), 2):
            res = self.activ(res @ self.weights[i] + self.weights[i + 1])

        return res

#==========================================================================================================

    def activ(self, res):
        """
        :param numpy_array res: output d'une couche
        :return l'activation LeakyReLu du resultat
        """
        res[res < 0] *= 0.0001
        return res

#==========================================================================================================

    def loss(self, res, y):
        nres = np.array(res, dtype=np.float64)
        ny = np.array(y, dtype=np.float64)
        if len(y.shape) == 1:
            ny = np.expand_dims(y, 1)
        if len(res.shape) == 1:
            nres = np.expand_dims(res, 1)
        return nres - ny

#==========================================================================================================

    def d_activ(self, res):
        """
        :param numpy_array res: output d'une couche
        :return la derive de l'activation LeakyReLu du resultat
        """
        dres = res
        dres[dres < 0] = 0.0001
        dres[dres >= 0] = 1
        return dres

#==========================================================================================================
    
    def forward(self, x, res_arr=None, dropping=False):
        """
        :param numpy_array x: input
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :return le resultat du forward et les resultats intermediaires
        """
        nx = np.array(x, dtype=np.float64)
        if res_arr:
            res_arr[0] = nx
        res = nx
        for h in range(0, len(self.weights), 2):  # calculate the result for the current features
#             print("res: ", res.dtype)
            res = self.activ(res @ self.weights[h] + self.weights[h + 1])
            if dropping:
                res = np.multiply(res, self.drop_out_layers[h//2])
                pass
            else:
                res = np.multiply(res, self.drop_prob)
                pass
            if res_arr:
                res_arr[h//2+1] = res
        
        if res_arr:
            return res, res_arr
        else:
            return res

#==========================================================================================================
    
    def backward(self, res, y, res_arr, batch_gradient, learning_rate, batch_size):
        """
        :param numpy_array res: resultat du forward
        :param numpy_array y: vraie resultat
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :param numpy_array batch_gradient: array contenant la somme des gradients calcules sur le batch
        :param integer float: learning rate
        :param integer batch_size: taille du batch
        """
        
        # calcul de l'erreur finale
        # TODO: adapter pour que ca fonctionne avec plusieurs outputs
        if len(y.shape) == 1:
            loss = res - np.expand_dims(y, 1)
        else:
            loss = res - y
        
        
        # update of the last neurone
        delta = loss * self.d_activ(res)
        batch_gradient[-2] = res_arr[-2].T @ delta
        batch_gradient[-1] += np.sum(delta)
        
        #         print("bg: ", batch_gradient[-2].shape)
        #         print("ra-2: ", res_arr[-2].T.shape)
        #         print("weights: ", self.weights[0].shape)
        #         print("delta: ", delta.shape)
        #         print("y: ", y.shape)
        #         print("res: ", res.shape)
        #         print("loss: ", loss.shape)
        #         print("bg: ", batch_gradient[-2])


        for b in reversed(range(0, len(self.weights) - 2, 2)):  # backpropagations
            l_output = res_arr[(b // 2) + 1]  # layer output
            loss = delta @ self.weights[b + 2].T
            delta = loss * self.d_activ(l_output)
            batch_gradient[b] = res_arr[b//2].T @ delta
            batch_gradient[b+1] += np.sum(delta)

        for k in range(len(batch_gradient)):
            self.weights[k] -= learning_rate * (1 / batch_size) * batch_gradient[k]

#==========================================================================================================

    def fit(self, x, y, step, epochs, batch_size, learning_rate, validation_datas):
        """
        :param numpy_array x: input
        :param numpy_array y: vraie resultat
        :param numpy_array res_arr: array ou on stock les resultats des couches intermediaires
        :param numpy_array batch_gradient: array contenant la somme des gradients calcules sur le batch
        :param integer float: learning rate
        :param integer batch_size: taille du batch
        """
        x_val = validation_datas[0]
        y_val = validation_datas[1]
        
        x = x
        y = y
        
        i = 0
        epo = 0
        steps_per_epoch = step // epochs

        default_batch_gradient = []
        for k in range(0, len(self.weights), 1):
            default_batch_gradient.append(np.zeros(self.weights[k].shape))
        
        default_res_arr = [np.zeros((self.weights[0].shape[0], batch_size))]
        for k in range(0, len(self.weights), 2):
            default_res_arr.append(np.zeros((self.weights[k].shape[1], batch_size)))
        
#         print(self.drop_out_layers)

        for etape in tqdm(range(step)):
            i += 1
            
            # recuperation des donnees du batch
            bx = x.iloc[batch_size * i : batch_size * (i + 1)].to_numpy()
            by = y.iloc[batch_size * i : batch_size * (i + 1)].to_numpy()
            
            # store the gradient calculated osn the whole batch
            batch_gradient = copy.deepcopy(default_batch_gradient)
            # store the result after each layer
            res_arr = copy.deepcopy(default_res_arr)
            
            self.drop_out_layers = []
            for k in range(0, len(self.weights), 2):
                self.drop_out_layers.append(np.random.random(self.weights[k+1].shape) < self.drop_prob)
            
            res, res_arr = self.forward(bx, res_arr, dropping=True)
            
            self.backward(res, by, res_arr, batch_gradient, learning_rate, batch_size)

            if x.shape[0] < batch_size * (i + 1) + 1:
                p = np.random.permutation(len(x))
                x = x.iloc[p]
                y = y.iloc[p]
                i = 0
                
            if etape != 0 and etape % (steps_per_epoch) == 0:
                predic_val = self.forward(x_val)[0]
                predic_train = self.forward(x.iloc[:step // epochs])
                print("Resume des erreurs sur la validation")
                print(pd.DataFrame(self.loss(predic_val, y_val)).describe())
                print("BG")
                print(batch_gradient[-2][:5])
                val_cost = ((self.loss(predic_val, y_val)) ** 2).mean()
                train_cost = ((self.loss(predic_train, y.iloc[:step // epochs])) ** 2).mean()
                
                val_rmse = math.sqrt(val_cost)
                train_rmse = math.sqrt(train_cost)
                
                print(
                    "Epoch: ",
                    epo,
                    "| loss val: ",
                    val_rmse,
                    " | loss train ",
                    train_rmse,
                )
                print("mean predic: ", predic_val.mean(), " / ", y_val.mean())
                
                epo += 1

# Recuperation des donnees

In [ ]:
train_dataframe = pd.read_csv("train.csv")
train_dataframe.describe()
train_dataframe = train_dataframe.reindex(
    np.random.permutation(train_dataframe.index))

In [ ]:
def preprocess_features(dataframe):
    """Prepares input features from Ames housing data set.

    Args:
    dataframe: A Pandas DataFrame expected to contain data
      from the Ames housing data set.
    Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
    """
    processed_features = dataframe.copy()
    
    processed_features['MSZoning'] = encode_and_bind(processed_features, 'MSZoning')
    processed_features['LotFrontage'] = encode_and_bind(processed_features, 'LotFrontage')
    processed_features['Street'] = encode_and_bind(processed_features, 'Street')
    processed_features['Alley'] = encode_and_bind(processed_features, 'Alley')
    processed_features['LotShape'] = encode_and_bind(processed_features, 'LotShape')
    processed_features['LandContour'] = encode_and_bind(processed_features, 'LandContour')
    processed_features['Utilities'] = encode_and_bind(processed_features, 'Utilities')
    processed_features['LotConfig'] = encode_and_bind(processed_features, 'LotConfig')
    processed_features['LandSlope'] = encode_and_bind(processed_features, 'LandSlope')
    processed_features['Neighborhood'] = encode_and_bind(processed_features, 'Neighborhood')
    processed_features['Condition1'] = encode_and_bind(processed_features, 'Condition1')
    processed_features['Condition2'] = encode_and_bind(processed_features, 'Condition2')
    processed_features['BldgType'] = encode_and_bind(processed_features, 'BldgType')
    processed_features['HouseStyle'] = encode_and_bind(processed_features, 'HouseStyle')
    processed_features['RoofStyle'] = encode_and_bind(processed_features, 'RoofStyle')
    processed_features['RoofMatl'] = encode_and_bind(processed_features, 'RoofMatl')
    processed_features['Exterior1st'] = encode_and_bind(processed_features, 'Exterior1st')
    processed_features['Exterior2nd'] = encode_and_bind(processed_features, 'Exterior2nd')
    processed_features['MasVnrType'] = encode_and_bind(processed_features, 'MasVnrType')
    processed_features['MasVnrArea'] = encode_and_bind(processed_features, 'MasVnrType')
    processed_features['ExterQual'] = encode_and_bind(processed_features, 'ExterQual')
    processed_features['ExterCond'] = encode_and_bind(processed_features, 'ExterCond')
    processed_features['Foundation'] = encode_and_bind(processed_features, 'Foundation')
    processed_features['BsmtQual'] = encode_and_bind(processed_features, 'BsmtQual')
    processed_features['BsmtCond'] = encode_and_bind(processed_features, 'BsmtCond')
    processed_features['BsmtExposure'] = encode_and_bind(processed_features, 'BsmtExposure')
    processed_features['BsmtFinType1'] = encode_and_bind(processed_features, 'BsmtFinType1')
    processed_features['BsmtFinType2'] = encode_and_bind(processed_features, 'BsmtFinType2')
    processed_features['Heating'] = encode_and_bind(processed_features, 'Heating')
    processed_features['HeatingQC'] = encode_and_bind(processed_features, 'HeatingQC')
    processed_features['CentralAir'] = encode_and_bind(processed_features, 'CentralAir')
    processed_features['Electrical'] = encode_and_bind(processed_features, 'Electrical')
    processed_features['KitchenQual'] = encode_and_bind(processed_features, 'KitchenQual')
    processed_features['Functional'] = encode_and_bind(processed_features, 'Functional')
    processed_features['FireplaceQu'] = encode_and_bind(processed_features, 'FireplaceQu')
    processed_features['GarageType'] = encode_and_bind(processed_features, 'GarageType')
    processed_features['GarageYrBlt'] = encode_and_bind(processed_features, 'GarageYrBlt')
    processed_features['GarageFinish'] = encode_and_bind(processed_features, 'GarageFinish')
    processed_features['GarageQual'] = encode_and_bind(processed_features, 'GarageQual')
    processed_features['GarageCond'] = encode_and_bind(processed_features, 'GarageCond')
    processed_features['PavedDrive'] = encode_and_bind(processed_features, 'PavedDrive')
    processed_features['PoolQC'] = encode_and_bind(processed_features, 'PoolQC')
    processed_features['Fence'] = encode_and_bind(processed_features, 'Fence')
    processed_features['MiscFeature'] = encode_and_bind(processed_features, 'MiscFeature')
    processed_features['SaleType'] = encode_and_bind(processed_features, 'SaleType')
    processed_features['SaleCondition'] = encode_and_bind(processed_features, 'SaleCondition')
    selected_features = processed_features#[['MSSubClass', 'MSZoning', 'SaleType', 'SaleCondition', 'LotArea', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                                          # 'Condition1', 'Condition2','BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
                                          #  'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
                                          #  'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
                                          #  'PoolQC', 'Fence', 'MiscFeature']]
    if 'SalePrice' in list(selected_features):
        selected_features = selected_features.drop('SalePrice', 1)
    # Create a synthetic feature.
#     processed_features["rooms_per_person"] = (
#      dataframe["total_rooms"] /
#      dataframe["population"])
    return selected_features

def preprocess_targets(dataframe):
    """Prepares target features (i.e., labels) from Ames housing data set.

    Args:
    dataframe: A Pandas DataFrame expected to contain data
    from the Ames housing data set.
    Returns:
    A DataFrame that contains the target feature.
    """
    output_targets = pd.DataFrame()
    # Scale the target to be in units of thousands of dollars.
    output_targets["SalePrice"] = (
      dataframe["SalePrice"] / 1000.0)
    return output_targets

In [ ]:
def my_one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [ ]:
# dataset = pd.read_csv("./winequality-red.csv")

# train = dataset
# validation = dataset.tail(199)

# x_train = train.drop('quality', 1)
# y_train = train.quality

# train_sata = pd.read_csv("./train_satander.csv")
# x_train = train_sata.drop("target", 1)
# x_train = x_train.drop("ID_code", 1)
# y_train = train_sata.target

cali_dataframe = pd.read_csv("./california_housing_train.csv")
# x = cali_dataframe.drop("median_house_value", 1)
# y = cali_dataframe.median_house_value / 100000

# wine_dataframe = pd.read_csv("./winequality-red.csv")
# x_train = wine_dataframe.drop("quality", 1)
# y_train = wine_dataframe.quality

mnist_dataframe = pd.read_csv("../kaggle/MNIST/train.csv")
x = mnist_dataframe.drop("label", 1) / 255
y = mnist_dataframe.label
# x.describe()

# Normalisation et transformation

In [ ]:
def normalize(x):
    return (x - min(x)) / (max(x) - min(x))


for k in x:
    x[k] = normalize(x[k])
    pass

x_train = x.head(15000)
y_train = y.head(15000)

x_val = x.tail(2000)
y_val = y.tail(2000)

# Choose the first 12000 (out of 17000) examples for training.
x_train = preprocess_features(train_dataframe.head(1200))
y_train = preprocess_targets(train_dataframe.head(1200))

# Choose the last 5000 (out of 17000) examples for validation.
x_val = preprocess_features(train_dataframe.tail(260))
y_val = preprocess_targets(train_dataframe.tail(260))

print(x_val.describe())

print("y summary")
print(y_train.describe())

# Training

In [ ]:
model = NN(x_train.shape[1], (100, 100, 100, 1), drop_prob=0.2)

In [ ]:
print("first predic: ", math.sqrt( ((model.loss(model.forward(x_val), y_val))**2).mean()) )
print("Shape: ", x_train.shape)
print("Training")
model.fit(
    x_train,
    y_train,
    step=300000,
    epochs=50,
    batch_size=50,
    learning_rate=0.00000000001,
    validation_datas=(x_val, y_val),
)

In [ ]:
model_save = copy.deepcopy(model)